# GPT-2 tuning for horoscopes using Sber GPT
Original git: https://github.com/sberbank-ai/

Открыть ноутбук можно на Colab:
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1TcDhTlre5nNDET-1kCe9hBR3-pBB6szt?usp=sharing)

In [ ]:
import regex as re
import os
from fastai.basics import *
from multiprocessing import Pool
#from colab_ssh import launch_ssh


In [ ]:
!git clone https://github.com/sberbank-ai/ru-gpts.git
%cd ru-gpts
!pip install -r requirements.txt

Cloning into 'ru-gpts'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 215 (delta 102), reused 108 (delta 33), pack-reused 0
Receiving objects: 100% (215/215), 190.18 KiB | 12.68 MiB/s, done.
Resolving deltas: 100% (102/102), done.
/content/ru-gpts
     |████████████████████████████████| 1.4MB 9.5MB/s 
     |████████████████████████████████| 1.1MB 52.7MB/s 
     |████████████████████████████████| 133kB 56.1MB/s 
     |████████████████████████████████| 696kB 53.6MB/s 
     |████████████████████████████████| 573kB 47.5MB/s 
     |████████████████████████████████| 5.9MB 49.2MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 890kB 19.6MB/s 
     |████████████████████████████████| 3.7MB 43.8MB/s 
     |████████████████████████████████| 552kB 51.4MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434678 sha256=e5a

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%set_env CUDA_HOME=/usr/local/cuda-10.1
!git clone https://github.com/NVIDIA/apex
%cd apex
# fix setup.py if complains for version mismatch
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

env: CUDA_HOME=/usr/local/cuda-10.1
Cloning into 'apex'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 7456 (delta 0), reused 0 (delta 0), pack-reused 7455
Receiving objects: 100% (7456/7456), 13.91 MiB | 27.83 MiB/s, done.
Resolving deltas: 100% (5036/5036), done.
/content/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-safxvi7s
Created temporary directory: /tmp/pip-req-tracker-gq0vj5og
Created requirements tracker '/tmp/pip-req-tracker-gq0vj5og'
Created temporary directory: /tmp/pip-install-qf3g08dy
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-3kgd5fq6
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-gq0vj5og'
    Running setup.py (pa

In [ ]:
DATASET_PATH = "/content/gdrive/My Drive/dataset"
SAVE_TO_PATH = "./dataset/prepared/"

In [ ]:
%cd ru-gpts

/content/ru-gpts


In [ ]:
def process_function(path_to_file):
    match = re.compile(r'(?=[^ ])([\W])([\w])')
    match2 = re.compile('(.|\s)\\1\\1+')
    with open(path_to_file, 'r') as f:
        lines = f.read()
    if lines and lines[0] != ' ': lines = ' ' + lines
    lines = match.sub(r'\g<1> \g<2>', lines)
    lines = match2.sub(r'\1'*3, lines)
    path = os.path.join(SAVE_TO_PATH, os.path.split(path_to_file)[1])
    with open(path, 'w') as handle:
        handle.write(lines)

In [ ]:
txts = get_files(DATASET_PATH, '.txt')
if not os.path.exists(SAVE_TO_PATH):
    os.makedirs(SAVE_TO_PATH)


for _ in progress_bar(Pool(64).imap_unordered(process_function, txts), len(txts)):
    pass

In [ ]:
!mkdir ./dataset/validation

In [ ]:
!mkdir ./dataset/train

In [ ]:
!mv ./dataset/prepared/train.txt ./dataset/train/


In [ ]:
!mv ./dataset/prepared/validation.txt ./dataset/validation

In [ ]:
%set_env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [ ]:
!ls

arguments.py	   generate_samples.py	     pretrain_transformers.py
configure_data.py  generate_transformers.py  README.md
dataset		   gpt_data_loader.py	     requirements.txt
data_utils	   learning_rates.py	     runs
detokenizer.py	   LICENSE		     scripts
evaluate_gpt.py    model		     utils.py
examples	   mpu
fp16		   pretrain_megatron.py


In [ ]:
!python3 pretrain_transformers.py \
    --output_dir ="/content/output" \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt2large \
    --do_train \
    --train_data_file=./dataset/train/train.txt \
    --do_eval \
    --eval_data_file=./dataset/validation/validation.txt \
    --fp16 \
    --block_size=128

2020-10-26 20:03:49.285481: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
10/26/2020 20:03:51 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
10/26/2020 20:03:52 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt2large/config.json from cache at /root/.cache/torch/transformers/ebe6992d0dca023ae6f51f91bf62ac9c23b25b6124d1ddd58b8bb2598a2da179.4b32e824394ba7bdd31d25b3d7541494f70fdb32554caf4178cd9ae8017976cc
10/26/2020 20:03:52 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": null,
  "attn_pdrop": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 50256,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "embd_pdrop": 0.1,
  "eos_to

Ждем решения вопроса с GPU. 